Zachary Proom

EN.605.646.81: Natural Language Processing

# Lab #3

## a

First, I split the training data in train.tsv into two groups, based on the class.

In [1]:
import pandas as pd

In [11]:
# Load in train.tsv.
training_data = pd.read_csv('train.tsv', sep = '\t', header = None) 

# Add column names.
training_data.columns = ["stars", "docid", "text"]

# Split into two groups based on class.
training_data_negative = training_data.loc[training_data["stars"] == 2]
training_data_positive = training_data.loc[training_data["stars"] == 4]

I found ten words that indicate positive or negative sentiment, and I show their relative frequencies in the table below (i.e. the percent of reviews they appear in in each class). The first five words indicate positive sentiment, and the last five indicate negative sentiment.

In [34]:
relative_freqs = pd.DataFrame(columns = ['word', 'positive', 'negative'])

for word in ["amazing", "awesome", "great", "incredible", "fantastic", "terrible", "horrible", "worst", "awful", "bad"]:
    negative_freq = sum(training_data_negative['text'].str.contains(word))/len(training_data_negative) * 100
    positive_freq = sum(training_data_positive['text'].str.contains(word))/len(training_data_positive) * 100
    relative_freqs = pd.concat([relative_freqs, pd.DataFrame({"word": [word], "positive": [positive_freq], "negative": [negative_freq]})], ignore_index = True)

relative_freqs = relative_freqs.reset_index(drop=True)
relative_freqs

,word,positive,negative
0,amazing,5.2,1.8
1,awesome,5.7,2.2
2,great,35.9,17.3
3,incredible,0.4,0.1
4,fantastic,3.7,1.0
5,terrible,0.3,3.2
6,horrible,0.6,3.1
7,worst,0.7,4.7
8,awful,0.6,2.5
9,bad,7.1,16.7


After reading a few of the reviews, I noticed that there's a lot of mixed language. Take this review as an example: 

2	YT9tezwopYagEjTxIzN2dg	i am just not a fan of this kind of pizza. i hate the sweet sauce, i hate that the ingredients are under the cheese so they don't get crunchy and crispy, the pepperoni is floppy. just not for me. the crust was kinda gooey like.   my delivery was 30 minutes late but its ok bc i wasn't in a hurry. they were really nice i just can't stand this kind of pizza.    if you like the sweet sauce and toppings under the cheese then go for it bc you'll probably love it!

The review is part of the negative class, but it contains several positive words such as "love", "nice", and "like". This is true for a lot of the reviews I read. A lot of the reviews are balanced and not polarized. This makes classifying sentiment more difficult.

## b